In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os

In [ ]:
!unzip "/content/gdrive/MyDrive/FinancialPhraseBank-v1.0.zip"

Archive:  /content/gdrive/MyDrive/FinancialPhraseBank-v1.0.zip
   creating: FinancialPhraseBank-v1.0/
  inflating: FinancialPhraseBank-v1.0/License.txt  
   creating: __MACOSX/
   creating: __MACOSX/FinancialPhraseBank-v1.0/
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._License.txt  
  inflating: FinancialPhraseBank-v1.0/README.txt  
  inflating: __MACOSX/FinancialPhraseBank-v1.0/._README.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_50Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_66Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_75Agree.txt  
  inflating: FinancialPhraseBank-v1.0/Sentences_AllAgree.txt  


In [ ]:
data = pd.read_csv('/content/FinancialPhraseBank-v1.0/Sentences_50Agree.txt', sep='@', names=['text','label'], engine='python', encoding='latin1')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4846 non-null   object
 1   label   4846 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


In [ ]:
df_unique = data.drop_duplicates(subset=['text'])
print(df_unique.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4838 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4838 non-null   object
 1   label   4838 non-null   object
dtypes: object(2)
memory usage: 113.4+ KB
None


In [ ]:
train, test = train_test_split(df_unique, test_size=0.2, random_state=0)
test = test.reset_index(drop=True)

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    968 non-null    object
 1   label   968 non-null    object
dtypes: object(2)
memory usage: 15.2+ KB


In [ ]:
test.label.value_counts()

label
neutral     592
positive    267
negative    109
Name: count, dtype: int64

In [ ]:
test.head()

,text,label
0,"Raute , headquartered in Nastola , Finland , i...",neutral
1,"And earlier this month , the company signed a ...",positive
2,The divested activities had net sales of EUR 1...,neutral
3,`` We can say that the number of deals has bec...,negative
4,The manager is critical of politicians ' failu...,neutral


# GPT-4 labelling

In [ ]:
!pip install openai
from openai import OpenAI
client = OpenAI(api_key='sk-proj-hEv32vgqXTnk8oxTyhhpT3BlbkFJSBixmeuoz6s5mkpOg9NU',)
import pandas as pd
import re  # Import the regex module

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.7 MB/s eta 0:00:00


In [ ]:
def create_merged_texts(gpt4):
  data = gpt4['text'].tolist()  # Convert the DataFrame column to a list for easier manipulation

  merged_texts = []  # This will store all merged text blocks

  # Process each block of 100 statements
  for i in range(0, len(data), 100):
      # Extract the slice for the current block
      block = data[i:i+100]
      # Merge the statements with numbering and new lines
      merged_text = '\n'.join(f"{j+1}. {s}" for j, s in enumerate(block))
      # Store the merged text in the list
      merged_texts.append(merged_text)

  return merged_texts

In [ ]:
def predict_financial_sentiment(user_prompt, max_tokens, model):
    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": "As a financial expert, analyze the potential immediate impact of financial news statements on the stock prices of the relevant companies. Assess whether each statement suggests a positive, negative, or neutral effect on the company’s stock price. Ensure that your analysis of each statement is conducted independently to avoid bias from other statements."
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        temperature=0.0, # Setting temperature to 0 for more deterministic outputs
        max_tokens=max_tokens,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    print(response)

    text_response = response.choices[0].message.content
    sentences = re.split(r'\d+\.\s', text_response)
    if sentences and sentences[0] == '':
        sentences = sentences[1:]

    # Strip leading and trailing whitespace and newline characters from each sentence
    sentences = [sentence.strip() for sentence in sentences]

    dataset = pd.DataFrame({
        'gpt_label': sentences
    })

    return dataset

In [ ]:
# Initialize an empty list to store DataFrames
def create_label(merged_texts, model):
  results = []

  for statement in merged_texts:
      user_prompt = f"Acting as a financial expert, analyze the following 100 numbered independent financial news statements, each pertaining to a different company. Assess the potential immediate impact on the stock price of the relevant company independently of any other statements. Determine whether the statement suggests a positive, negative, or neutral immediate effect on the company’s stock price. Consider factors such as tone, relevance of information, and market context in your evaluation. Please only provide potential immediate impacts (positive, negative, or neutral) for each of the statements in a numbered format, with each prediction corresponding to the order of the statements provided.\n{statement}"
      try:
          # Append the result of each prediction to the results list
          results.append(predict_financial_sentiment(user_prompt, 600, model=model))
      except Exception as e:
          print(f"Error processing statement: {statement}\nError: {e}")

  return results


In [ ]:
merged_texts = create_merged_texts(test)
results = create_label(merged_texts,"gpt-3.5-turbo")
df = pd.concat(results, ignore_index=True)
df.loc[:, 'gpt_label'] = df['gpt_label'].str.lower()
df = df.rename(columns={'gpt_label': 'gpt35_label'})
# Combine the original data with the predictions
test_labelled = pd.concat([test,df.reset_index(drop=True)], axis=1)
print(test.isna().any().any())

ChatCompletion(id='chatcmpl-9ao8JxRcXNDz9R5oOO8l9CCNybDzA', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Neutral\n2. Positive\n3. Positive\n4. Neutral\n5. Neutral\n6. Neutral\n7. Positive\n8. Negative\n9. Neutral\n10. Neutral\n11. Negative\n12. Neutral\n13. Neutral\n14. Neutral\n15. Neutral\n16. Neutral\n17. Neutral\n18. Negative\n19. Positive\n20. Neutral\n21. Neutral\n22. Neutral\n23. Neutral\n24. Neutral\n25. Positive\n26. Positive\n27. Positive\n28. Neutral\n29. Neutral\n30. Negative\n31. Neutral\n32. Neutral\n33. Neutral\n34. Neutral\n35. Neutral\n36. Negative\n37. Negative\n38. Neutral\n39. Negative\n40. Neutral\n41. Positive\n42. Neutral\n43. Neutral\n44. Neutral\n45. Neutral\n46. Neutral\n47. Neutral\n48. Neutral\n49. Positive\n50. Neutral\n51. Neutral\n52. Positive\n53. Positive\n54. Negative\n55. Neutral\n56. Negative\n57. Neutral\n58. Neutral\n59. Neutral\n60. Neutral\n61. Neutral\n62. Negative\n63. Positive\n64. Neu

In [ ]:
test_labelled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         968 non-null    object
 1   label        968 non-null    object
 2   gpt35_label  968 non-null    object
dtypes: object(3)
memory usage: 22.8+ KB


In [ ]:
test_labelled.head()

,text,label,gpt35_label
0,"Raute , headquartered in Nastola , Finland , i...",neutral,neutral
1,"And earlier this month , the company signed a ...",positive,positive
2,The divested activities had net sales of EUR 1...,neutral,positive
3,`` We can say that the number of deals has bec...,negative,neutral
4,The manager is critical of politicians ' failu...,neutral,neutral


In [ ]:
test_labelled.to_csv('/content/gdrive/MyDrive/GPT35predictionsonPhraseBankTest.csv', index=False)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

def evaluate_model_performance(df, actual_col, predicted_col):
    """
    Evaluates the model performance based on provided DataFrame and column names.

    Parameters:
    - df (pd.DataFrame): The DataFrame containing the data.
    - actual_col (str): The name of the column containing the actual labels.
    - predicted_col (str): The name of the column containing the predicted labels.

    Returns:
    - dict: A dictionary containing the accuracy, F1 score, precision, and recall.
    """
    # Check if the specified columns exist in the DataFrame
    if actual_col not in df.columns or predicted_col not in df.columns:
        raise ValueError("Specified columns are not in the DataFrame")

    # Extract actual and predicted labels from the DataFrame
    actual_labels = df[actual_col]
    predicted_labels = df[predicted_col]

    # Calculate metrics
    accuracy = accuracy_score(actual_labels, predicted_labels)
    f1 = f1_score(actual_labels, predicted_labels, average='weighted')
    precision = precision_score(actual_labels, predicted_labels, average='weighted')
    recall = recall_score(actual_labels, predicted_labels, average='weighted')

    # Return the results as a dictionary
    results = {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall
    }
    return results

In [ ]:
results = evaluate_model_performance(test_labelled, 'label', 'gpt35_label')
print(results)

{'Accuracy': 0.731404958677686, 'F1 Score': 0.7366287890886019, 'Precision': 0.7613720325627471, 'Recall': 0.731404958677686}


In [ ]:
merged_texts = create_merged_texts(test_labelled)
results = create_label(merged_texts,"gpt-4o")
df = pd.concat(results, ignore_index=True)
df.loc[:, 'gpt_label'] = df['gpt_label'].str.lower()
df = df.rename(columns={'gpt_label': 'gpt4o_label'})
# Combine the original data with the predictions
test_labelled = pd.concat([test_labelled,df.reset_index(drop=True)], axis=1)
print(test_labelled.isna().any().any())

ChatCompletion(id='chatcmpl-9aoARRsaJnI7JoR2hDnswYe6mEIia', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='1. Neutral\n2. Positive\n3. Positive\n4. Negative\n5. Neutral\n6. Neutral\n7. Positive\n8. Negative\n9. Neutral\n10. Neutral\n11. Negative\n12. Negative\n13. Positive\n14. Neutral\n15. Neutral\n16. Neutral\n17. Neutral\n18. Negative\n19. Positive\n20. Neutral\n21. Positive\n22. Neutral\n23. Neutral\n24. Positive\n25. Positive\n26. Positive\n27. Positive\n28. Neutral\n29. Neutral\n30. Negative\n31. Positive\n32. Negative\n33. Neutral\n34. Positive\n35. Neutral\n36. Negative\n37. Negative\n38. Neutral\n39. Negative\n40. Neutral\n41. Positive\n42. Positive\n43. Neutral\n44. Positive\n45. Positive\n46. Positive\n47. Neutral\n48. Neutral\n49. Positive\n50. Negative\n51. Neutral\n52. Positive\n53. Positive\n54. Negative\n55. Positive\n56. Negative\n57. Neutral\n58. Neutral\n59. Positive\n60. Neutral\n61. Positive\n62. Negative\n63. P

In [ ]:
test_labelled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 968 entries, 0 to 967
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         968 non-null    object
 1   label        968 non-null    object
 2   gpt35_label  968 non-null    object
 3   gpt4o_label  968 non-null    object
dtypes: object(4)
memory usage: 30.4+ KB


In [ ]:
test_labelled.head()

,text,label,gpt35_label,gpt4o_label
0,"Raute , headquartered in Nastola , Finland , i...",neutral,neutral,neutral
1,"And earlier this month , the company signed a ...",positive,positive,positive
2,The divested activities had net sales of EUR 1...,neutral,positive,positive
3,`` We can say that the number of deals has bec...,negative,neutral,negative
4,The manager is critical of politicians ' failu...,neutral,neutral,neutral


In [ ]:
results = evaluate_model_performance(test_labelled, 'label', 'gpt4o_label')
print(results)

{'Accuracy': 0.743801652892562, 'F1 Score': 0.7467691978225061, 'Precision': 0.8042645932791511, 'Recall': 0.743801652892562}


In [ ]:
test_labelled.to_csv('/content/gdrive/MyDrive/GPTpredictionsonPhraseBankTest_1606.csv', index=False)